In [1]:
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Matrix)
library(parallel)
library(foreach)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDupl

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/5.merge_peaks_from_5_super_groups.240524/pairs.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/5.merge_peaks_from_5_super_groups.240524/matrix.atac.5_super_groups.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/5.merge_peaks_from_5_super_groups.240524/matrix.rna.5_super_groups.rds"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/Signac/BMMC5.merge.250210/"

Import candidate E-G pairs

In [3]:
pairs.E2G = readRDS(path.pairs.E2G)

Import ATAC matrix

In [4]:
matrix.atac = readRDS(path.matrix.atac_count)

Import RNA matrix

In [5]:
matrix.rna_count = readRDS(path.matrix.rna_count)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

Map gene names

In [6]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
    library(dplyr)
    library(data.table)
    
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [7]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:Biostrings’:

    collapse, intersect, setdiff, setequal, union


The following object is masked from ‘package:XVector’:

    slice


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, 

Prepare background data

In [8]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak = bed.peak[seqnames(bed.peak) %in% paste("chr",c(1:22,"X","Y"), sep = "")]
bed.peak

GRanges object with 303482 ranges and 0 metadata columns:
           seqnames              ranges strand
              <Rle>           <IRanges>  <Rle>
       [1]     chr1 100015450-100016026      *
       [2]     chr1 100028395-100029373      *
       [3]     chr1 100037560-100039288      *
       [4]     chr1 100046092-100047273      *
       [5]     chr1 100047388-100047636      *
       ...      ...                 ...    ...
  [303478]     chrX     9987345-9987562      *
  [303479]     chrX     9988628-9989100      *
  [303480]     chrX       999358-999880      *
  [303481]     chrX     9995615-9996434      *
  [303482]     chrX     9997010-9998101      *
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [9]:
meta.features = RegionStats(bed.peak,
                            genome = BSgenome.Hsapiens.UCSC.hg38)
rownames(meta.features) = paste(bed.peak@seqnames,bed.peak@ranges,sep = "-")
meta.features$seqnames = as.character(bed.peak@seqnames)
meta.features = cbind(meta.features,
                      FindTopFeatures(matrix.atac)[rownames(meta.features),])
meta.features

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,⋯,GT,TA,TC,TG,TT,GC.percent,sequence.length,seqnames,count,percentile
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>
chr1-100015450-100016026,78,17,51,59,28,13,1,34,40,14,⋯,33,60,31,35,56,32.75563,577,chr1,161,0.15947239
chr1-100028395-100029373,81,53,77,61,78,64,10,72,59,44,⋯,49,54,62,66,100,43.41164,979,chr1,2172,0.87496787
chr1-100037560-100039288,104,79,102,58,85,162,135,120,93,157,⋯,96,62,104,108,98,58.58878,1729,chr1,26802,0.98844083
chr1-100046092-100047273,54,31,68,87,56,48,11,97,53,47,⋯,93,77,86,114,206,38.74788,1182,chr1,2612,0.89378612
chr1-100047388-100047636,10,9,11,16,15,15,10,18,15,20,⋯,10,6,14,24,39,48.19277,249,chr1,378,0.48796963
chr1-100048158-100048921,43,38,41,42,54,48,6,81,31,42,⋯,39,35,62,65,97,44.63351,764,chr1,542,0.60648737
chr1-100049442-100049773,14,18,18,17,19,31,5,37,17,20,⋯,22,17,23,36,29,47.89157,332,chr1,224,0.28060972
chr1-100050175-100050627,45,22,34,33,21,18,4,31,34,13,⋯,24,34,22,33,49,39.95585,453,chr1,278,0.36805807
chr1-100051282-100052434,97,50,81,97,72,43,8,70,59,43,⋯,71,97,57,84,175,35.90633,1153,chr1,890,0.73281447


Run Signac prediction

In [10]:
PredictionSignac = function(pairs.E2G,
                            data.RNA,
                            data.ATAC,
                            meta.features,
                            n_sample = 200,
                            cor_method = qlcMatrix::corSparse,
                            n.core = 16){
  
  
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (gene.tmp = unique(pairs.E2G$TargetGene),
             .combine = 'c',
             .packages = c("Signac",
                           "Matrix")) %dopar% {
                             
                             pairs.E2G.tmp = pairs.E2G[pairs.E2G$TargetGene == gene.tmp]
                             
                             data.RNA.tmp.t = t(data.RNA[gene.tmp, 
                                                         , drop = FALSE])
                             data.ATAC.tmp.t = t(data.ATAC[pairs.E2G.tmp$PeakName, 
                                                           , drop = FALSE])
                             
                             coef.result <- cor_method(X = data.ATAC.tmp.t, 
                                                       Y = data.RNA.tmp.t)
                             
                             
                             meta.features.bg <- meta.features[!meta.features$seqnames %in% as.character(seqnames(pairs.E2G.tmp)), ]
                             meta.features.peaks <- meta.features[pairs.E2G.tmp$PeakName, ]
                             bg.peaks <- lapply(X = seq_len(length.out = nrow(x = meta.features.peaks)), 
                                                FUN = function(x) {
                                                  MatchRegionStats(meta.feature = meta.features.bg, 
                                                                   query.feature = meta.features.peaks[x, , drop = FALSE], 
                                                                   features.match = c("GC.percent", "count", "sequence.length"), 
                                                                   n = n_sample, 
                                                                   verbose = FALSE)
                                                })
                             data.ATAC.bg.t <- t(data.ATAC[unlist(x = bg.peaks),, 
                                                           drop = FALSE])
                             bg.coef <- cor_method(X = data.ATAC.bg.t, Y = data.RNA.tmp.t)
                             rownames(bg.coef) <- colnames(data.ATAC.bg.t)
                             for (j in seq(length(pairs.E2G.tmp))) {
                               coef.use <- bg.coef[(((j - 1) * n_sample) + 
                                                      1):(j * n_sample), ]
                               z <- (coef.result[j] - mean(x = coef.use))/sd(x = coef.use)
                               pairs.E2G.tmp[j]$zscore <- z
                             }
                             pairs.E2G.tmp$pvalue = 1 - pnorm(pairs.E2G.tmp$zscore)
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [11]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna.rename) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]
mcols(pairs.E2G.filter)[,c("zscore","pvalue")] = NA
pairs.E2G.filter

GRanges object with 5698963 ranges and 5 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100015450-100016026      * |         AGL
        [2]     chr1 100015450-100016026      * |      CDC14A
        [3]     chr1 100015450-100016026      * |         DBT
        [4]     chr1 100015450-100016026      * |       EXTL2
        [5]     chr1 100015450-100016026      * |     MFSD14A
        ...      ...                 ...    ... .         ...
  [5698959]     chrX     9995615-9996434      * |        WWC3
  [5698960]     chrX     9997010-9998101      * |       CLCN4
  [5698961]     chrX     9997010-9998101      * |     SHROOM2
  [5698962]     chrX     9997010-9998101      * |       TBL1X
  [5698963]     chrX     9997010-9998101      * |        WWC3
                          PeakName               PairName    zscore    pvalue
                       <character>            <character>

In [12]:
start_time <- Sys.time()
pairs.E2G.res = PredictionSignac(pairs.E2G.filter,
                                 matrix.rna.rename,
                                 matrix.atac,
                                 meta.features,
                                 8)
end_time <- Sys.time()
execution_time <- end_time - start_time
execution_time

Time difference of 12.49306 hours

In [13]:
execution_time

Time difference of 12.49306 hours

Save results

In [14]:
dir.create(dir.output,recursive = T)
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

Warning message in dir.create(dir.output, recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/Signac/BMMC5.merge.250210' already exists”


GRanges object with 5698963 ranges and 5 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100015450-100016026      * |         AGL
        [2]     chr1 100028395-100029373      * |         AGL
        [3]     chr1 100037560-100039288      * |         AGL
        [4]     chr1 100046092-100047273      * |         AGL
        [5]     chr1 100047388-100047636      * |         AGL
        ...      ...                 ...    ... .         ...
  [5698959]     chrX   97527775-97528023      * |      DIAPH2
  [5698960]     chrX   97529488-97530051      * |      DIAPH2
  [5698961]     chrX   97563263-97564917      * |      DIAPH2
  [5698962]     chrX   97597862-97598279      * |      DIAPH2
  [5698963]     chrX   97617017-97617513      * |      DIAPH2
                          PeakName               PairName    zscore     pvalue
                       <character>            <character

In [15]:
df.output = as.data.frame(pairs.E2G.res)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,width,strand,TargetGene,PeakName,PairName,zscore,pvalue,CellType
<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
chr1,100015450,100016026,577,*,AGL,chr1-100015450-100016026,chr1-100015450-100016026_AGL,1.04470861,0.1480788322,K562
chr1,100028395,100029373,979,*,AGL,chr1-100028395-100029373,chr1-100028395-100029373_AGL,2.39148802,0.0083901159,K562
chr1,100037560,100039288,1729,*,AGL,chr1-100037560-100039288,chr1-100037560-100039288_AGL,2.93439727,0.0016709808,K562
chr1,100046092,100047273,1182,*,AGL,chr1-100046092-100047273,chr1-100046092-100047273_AGL,0.30908974,0.3786266319,K562
chr1,100047388,100047636,249,*,AGL,chr1-100047388-100047636,chr1-100047388-100047636_AGL,-1.46609757,0.9286891461,K562
chr1,100048158,100048921,764,*,AGL,chr1-100048158-100048921,chr1-100048158-100048921_AGL,-1.65681567,0.9512216256,K562
chr1,100049442,100049773,332,*,AGL,chr1-100049442-100049773,chr1-100049442-100049773_AGL,0.21353069,0.4154565286,K562
chr1,100050175,100050627,453,*,AGL,chr1-100050175-100050627,chr1-100050175-100050627_AGL,-1.73083461,0.9582593662,K562
chr1,100051282,100052434,1153,*,AGL,chr1-100051282-100052434,chr1-100051282-100052434_AGL,0.04020108,0.4839664099,K562


In [16]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] data.table_1.15.2                 dplyr_1.1.4                      
 [3] foreach_1.5.2                     Matrix_1.6